In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))


In [2]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 45000
Test data size: 10000
Validation data size: 5000


In [3]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

input_dim = (227, 227, 3)

input_img = Input(shape=input_dim)

# Layer 1
cl1 = Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu', 
             input_shape=input_dim)(input_img)
bnl2 = BatchNormalization()(cl1)
pl3 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(bnl2)

# Layer 2
cl4 = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(pl3)
bnl5 = BatchNormalization()(cl4)
pl6 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(bnl5)

# Layer 3
cl7 = Conv2D(filters=384, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu')(pl3)

# Layer 4
cl8 = Conv2D(filters=384, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu')(cl7)

# Layer 5
cl9 = Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu')(cl8)
pl10 = MaxPooling2D(pool_size=(3,3), strides=(2,2))(cl9)

# Layer 6
fl11 = Flatten()(pl10)
dl12 = Dense(4096, activation='relu')(fl11)
dol13 = Dropout(0.5)(dl12)
dl14 = Dense(4096, activation='relu')(dol13)
dol15 = Dropout(0.5)(dl14)
output = Dense(units=10, activation = 'softmax')(dol15)

classifier = Model(input_img, output)
opt = RMSprop(learning_rate=0.001)
# opt = Adam(learning_rate=0.01)


classifier.compile(optimizer = opt, loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])

print(classifier.summary())# Fitting the CNN to the images

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,  
                              min_delta=1e-4, mode='min', verbose=1)

stop_alg = EarlyStopping(monitor='val_loss', patience=35, 
                         restore_best_weights=True, verbose=1)

hist = classifier.fit(train_ds, batch_size=100, epochs=1000, 
                   callbacks=[stop_alg, reduce_lr], shuffle=True, 
                   validation_data=validation_ds,
                     validation_freq=1)

classifier.save_weights("alexnet.hdf5")

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 384)       332160    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 27, 27, 256)      


KeyboardInterrupt

